In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df_source = pd.read_csv("county-elections-2020.csv")
df_source.dropna(subset=["dem_ratio"], inplace=True)
df_source

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,dem_ratio,county-state
15,2012,SOUTH DAKOTA,SD,SHANNON,46113.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,2937,3145,20220315,TOTAL,0.933863,SHANNON - SD
16,2020,RHODE ISLAND,RI,FEDERAL PRECINCT,NaN,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,1276,1374,20220315,TOTAL,0.928675,FEDERAL PRECINCT - RI
17,2008,DISTRICT OF COLUMBIA,DC,DISTRICT OF COLUMBIA,11001.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,245800,265853,20220315,TOTAL,0.924571,DISTRICT OF COLUMBIA - DC
18,2020,DISTRICT OF COLUMBIA,DC,DISTRICT OF COLUMBIA,11001.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,317323,344356,20220315,TOTAL,0.921497,DISTRICT OF COLUMBIA - DC
19,2012,DISTRICT OF COLUMBIA,DC,DISTRICT OF COLUMBIA,11001.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,267070,293764,20220315,TOTAL,0.909131,DISTRICT OF COLUMBIA - DC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20901,2020,VIRGINIA,VA,WYTHE,51197.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,0,15073,20220315,PROVISIONAL,0.000000,WYTHE - VA
20902,2020,VIRGINIA,VA,BUENA VISTA CITY,51530.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,0,2776,20220315,PROVISIONAL,0.000000,BUENA VISTA CITY - VA
20903,2020,VIRGINIA,VA,COVINGTON CITY,51580.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,0,2603,20220315,PROVISIONAL,0.000000,COVINGTON CITY - VA
20904,2020,VIRGINIA,VA,EMPORIA CITY,51595.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,0,2384,20220315,PROVISIONAL,0.000000,EMPORIA CITY - VA


In [3]:
counties = df_source["county-state"].unique()
len(counties)

3193

In [4]:
df_counties = pd.DataFrame.from_dict({"county": counties})
df_counties

,county
0,SHANNON - SD
1,FEDERAL PRECINCT - RI
2,DISTRICT OF COLUMBIA - DC
3,BRONX - NY
4,PETERSBURG - VA
...,...
3188,KING - TX
3189,HANSFORD - TX
3190,OCHILTREE - TX
3191,GLASSCOCK - TX


In [5]:
YEARS = [2000, 2004, 2008, 2012, 2016, 2020]
YEARS_DICT = dict(zip(np.arange(len(YEARS)), YEARS))
YEARS_DICT

{0: 2000, 1: 2004, 2: 2008, 3: 2012, 4: 2016, 5: 2020}

In [6]:
df_source[(df_source['county-state'] == 'KING - TX') & (df_source['year'] == 2016)]['dem_ratio'].values[0]

0.031446541

In [29]:
def county_year_query(county: str, year: int) -> float | None:
    return df_source[
        (df_source["county-state"] == county) & (df_source["year"] == year)
    ]


for year in YEARS:
    dem_ratios = []
    for county in tqdm(df_counties["county"]):
        query_result = county_year_query(county, year)

        candidate_votes = sum(query_result["candidatevotes"])
        total_votes = sum(query_result["totalvotes"])
        if total_votes == 0:
            dem_ratio = 0
        else:
            dem_ratio = candidate_votes / total_votes

        # try:
        #     dem_ratio = sum(query_result["candidatevotes"]) / sum(
        #         query_result["totalvotes"]
        #     )
        # except ZeroDivisionError:
        #     dem_ratio = None

        dem_ratios.append(dem_ratio)
    df_counties[f"dr_{year}"] = dem_ratios

100%|██████████| 3193/3193 [00:02<00:00, 1509.25it/s]


In [30]:
df_counties

,county,dr_2000,dr_2004,dr_2008,dr_2012,dr_2016,dr_2020
0,SHANNON - SD,0.853559,0.846227,0.896500,0.933863,0.000000,0.000000
1,FEDERAL PRECINCT - RI,0.000000,0.000000,0.000000,0.804805,0.875000,0.928675
2,DISTRICT OF COLUMBIA - DC,0.851551,0.891839,0.924571,0.909131,0.908638,0.921497
3,BRONX - NY,0.862856,0.807567,0.887158,0.902145,0.885173,0.821866
4,PETERSBURG - VA,0.791087,0.810277,0.886358,0.897850,0.871972,0.000000
...,...,...,...,...,...,...,...
3188,KING - TX,0.102190,0.115385,0.049080,0.034483,0.031447,0.050314
3189,HANSFORD - TX,0.094828,0.111784,0.114177,0.081040,0.087827,0.081094
3190,OCHILTREE - TX,0.084740,0.079005,0.078160,0.084531,0.091272,0.095691
3191,GLASSCOCK - TX,0.068301,0.082552,0.093357,0.076125,0.056291,0.059724


In [44]:
def calculate_trend(row) -> float:
    weights = np.linspace(start=0.5, stop=1, endpoint=True, num=len(YEARS))
    year_columns = [f"dr_{year}" for year in YEARS]

    trend = 1

    for year_idx, year in YEARS_DICT.items():
        if year_idx == 0:
            continue

        if (row[year_columns[year_idx]] == 0) | (row[year_columns[year_idx - 1]] == 0):
            continue

        diff = row[year_columns[year_idx]] - row[year_columns[year_idx - 1]]
        trend += weights[year_idx] * diff
    
    return trend

In [45]:
trend_score = [calculate_trend(row) for row_idx, row in tqdm(df_counties.iterrows())]
df_counties["trend_score"] = trend_score
df_counties

3193it [00:00, 23594.35it/s]


,county,dr_2000,dr_2004,dr_2008,dr_2012,dr_2016,dr_2020,trend_score
0,SHANNON - SD,0.853559,0.846227,0.896500,0.933863,0.000000,0.000000,1.060683
1,FEDERAL PRECINCT - RI,0.000000,0.000000,0.000000,0.804805,0.875000,0.928675,1.116851
2,DISTRICT OF COLUMBIA - DC,0.851551,0.891839,0.924571,0.909131,0.908638,0.921497,1.047149
3,BRONX - NY,0.862856,0.807567,0.887158,0.902145,0.885173,0.821866,0.955948
4,PETERSBURG - VA,0.791087,0.810277,0.886358,0.897850,0.871972,0.000000,1.050674
...,...,...,...,...,...,...,...,...
3188,KING - TX,0.102190,0.115385,0.049080,0.034483,0.031447,0.050314,0.965961
3189,HANSFORD - TX,0.094828,0.111784,0.114177,0.081040,0.087827,0.081094,0.984715
3190,OCHILTREE - TX,0.084740,0.079005,0.078160,0.084531,0.091272,0.095691,1.011550
3191,GLASSCOCK - TX,0.068301,0.082552,0.093357,0.076125,0.056291,0.059724,0.987911


In [46]:
df_counties.to_csv("democrat-ratio-by-county-2020.csv")

In [ ]:
import plotly.express as px
px.scatter(data_frame=df_counties, x='dr_2020', y='trend_score', hover_data='county')